# Advent of Code 2020, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202020%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL. Why? A language that doesn't affect the way you think about programming is not worth knowing.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [75]:
⍝ Helper functions and common settings
⎕FR ⎕PP ⎕IO←1287 34 0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
rg←{⊃,/⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)}⊢⍵} ⍝ capture groups as vector
'segs'⎕CY'dfns'

⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→─────┐
│Was ON│
└──────┘

### Day 1: Report Repair
https://adventofcode.com/2020/day/1

Outer product sum, find locations of 2020 and multiply. 

In [50]:
⊢result←1 2{×/⍵[⊃⍸2020=∘.+⍣⍺⍨⍵]}¨⊂⍎⍕⊃⎕NGET'data/2020/day01.txt'1
assert 73371 127642310≡result

┌→──────────────┐
│73371 127642310│
└~──────────────┘

### Day 2: Password Philosophy
https://adventofcode.com/2020/day/2

In [40]:
⎕IO←1
DAY02←⍎¨@1 2¨'- :'∘segs¨⊃⎕NGET'data/2020/day02.txt'1

In [42]:
⊢result←+⌿↑{(b t l s)←⍵⋄((b∘≤∧t∘≥)+/l=s)(⊃(l=b⊃s)≠l=t⊃s)}¨DAY02
assert 528 497≡result

┌→──────┐
│528 497│
└~──────┘

Ok, perhaps a bit messy. Here's the same idea, in long-hand:

In [43]:
]dinput
Part1←{
    (min max letter string)←⍵
    count←+/letter=string
    (min∘≤∧≤∘max) count
}

In [47]:
]dinput
Part2←{
    (min max letter string)←⍵                ⍝ De-structure 
    ⊃(letter=min⊃string)≠(letter=max⊃string) ⍝ Pick the values, and XOR
}

In [45]:
assert 528=+/Part1¨DAY02
assert 497=+/Part2¨DAY02  

Another neat trick worth remembering is that the built-in ⎕CSV can do numeric conversion:

In [68]:
b t l s←↓⍉⎕CSV('\W+'⎕R','⊃⎕NGET'data/2020/day02.txt' 1)''4

Here's another version, jazzed up by ninja-master @ngn:

In [46]:
p q l s←↓⍉↑DAY02
+/(p∘≤∧≤∘q)+/¨s=∊l
+/≠⌿↑p q⊃¨¨⊂s=∊l

⍝ Or as a matrix
b←↑s=∊l
+/(p∘≤∧≤∘q)+/b
+/≠/(p,⍪q)⊃⍤0 1⍤1⊢b

528

497

528

497

### Day 3: Toboggan Trajectory
https://adventofcode.com/2020/day/3

The first approach that comes to mind is a reduce over the path's steps which we can implement either as a tail-recursive function, or by pre-calculating the coordinate vector.

In [61]:
⎕IO←0
DAY03←↑⊃⎕NGET'data/2020/day03.txt'1

In [62]:
]dinput
Slope←{
    (dy dx)←⍵
    d←⍺
    0 { ⍝ Tail-recursive accumulation
        (y x)←⍵
        y≥≢d:⍺
        (⍺+'#'=d[y;(1⊃⍴d)|x])∇dy dx + y x
    } 0 0
}

In [63]:
⊢result←DAY03∘Slope¨(1 1)(1 3)(1 5)(1 7)(2 1)

┌→──────────────┐
│68 203 78 77 40│
└~──────────────┘

In [64]:
assert 203=1⊃result ⍝ Part 1
assert 3316272960=×/result

Considerably more idiomatically, we can simply create the coordinate vector representing the slope first.

In [66]:
]dinput
Slope2←{
    coords←↓⍉↑(⍴⍺)|⍵×⊂⍳⌈(≢⍺)÷⊃⍵
    +/'#'=⍺[coords]
}

In [67]:
⊢result←DAY03∘Slope2¨(1 1)(1 3)(1 5)(1 7)(2 1)

┌→──────────────┐
│68 203 78 77 40│
└~──────────────┘

### Day 4: Passport Processing
https://adventofcode.com/2020/day/4

In [94]:
DAY04←↑,/¨{data⊂⍨1,2≠/⍵}0≠≢¨data←' :'∘segs¨⊃⎕NGET'data/2020/day04.txt'1
eyr←(2020∘≤∧≤∘2030)⍎
iyr←(2010∘≤∧≤∘2020)⍎
byr←(1920∘≤∧≤∘2002)⍎
ecl←'amb' 'blu' 'brn' 'gry' 'grn' 'hzl' 'oth'∊⍨⊂
pid←≢'^\d{9}$'⎕S''
hcl←≢'^#[a-f0-9]{6}$'⎕S''
cid←{1}
hgt←≢'^(((59|6[0-9]|7[0-6])in)|((1[5-8][0-9]|19[0-3])cm))$'⎕S''

In [96]:
part1←'byr' 'iyr' 'eyr' 'hgt' 'hcl' 'ecl' 'pid'∘(∧/∊)¨pp←(0≠≢¨DAY04)/DAY04
part2←{∧/{(⍎⍺)⍵}/(2÷⍨≢⍵)2⍴⍵}¨part1/pp
⊢result←+/¨part1 part2
assert 256 198≡result

┌→──────┐
│256 198│
└~──────┘

### Day 5: Binary Boarding
https://adventofcode.com/2020/day/5

In [97]:
DAY05←⊃⎕NGET'data/2020/day05.txt'1

In [105]:
SID←{0=≢⍵:⊃⍺⋄(lo hi)←⍺⋄half←hi÷2⋄(0⊃⍵)∊'FL':(lo,half)∇1↓⍵⋄((lo+half),half)∇1↓⍵}

In [106]:
seats←0 1024∘SID¨DAY05
⊢part1←⌈/seats
assert 888=part1

888

In [107]:
sorted←seats[⍋seats]
gap←⍸~2{⍵=⍺+1}/sorted
⊢part2←1+gap⊃sorted
assert 522=part2

522

A smarter solution for part 1, courtesy of @rak1507:

In [111]:
⍝ Part 1
⌈/seats←2⊥'BR'∊⍨⍉↑DAY05

888

### Day 6: Custom Customs
https://adventofcode.com/2020/day/6

In [115]:
DAY06←{⍵⊆⍨(⊂'')≢¨⍵}⊃⎕NGET'data/2020/day06.txt'1

In [118]:
⊢part1←≢∊∪/¨DAY06
assert 6416=part1

6416

In [119]:
⊢part2←≢∊∩/¨DAY06
assert 3050=part2

3050